In [14]:
 !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
#importing all libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
import numpy as np


In [ ]:
tweet = pd.read_csv('elonmusk.csv')
tweet.head()

,Date Created,Number of Likes,Source of Tweet,Tweets
0,2022-09-12 05:44:11+00:00,1524,Twitter for iPhone,@teslaownersSV @cb_doge @Tesla @mayemusk I gue...
1,2022-09-12 05:43:02+00:00,19631,Twitter for iPhone,@cb_doge @Tesla @mayemusk Still doing same thi...
2,2022-09-12 04:19:57+00:00,9221,Twitter for iPhone,Looks good to roll out to all Tesla owners wit...
3,2022-09-12 03:25:03+00:00,944,Twitter for iPhone,@Tesla__Mania @WholeMarsBlog That is probably ...
4,2022-09-12 01:48:49+00:00,3710,Twitter for iPhone,@WholeMarsBlog Real-world validation &amp; bil...


In [ ]:
#from datetime we extract date feature as well
tweet['date'] = pd.to_datetime(tweet['Date Created']).dt.date
tweet.head()

,Date Created,Number of Likes,Source of Tweet,Tweets,date
0,2022-09-12 05:44:11+00:00,1524,Twitter for iPhone,@teslaownersSV @cb_doge @Tesla @mayemusk I gue...,2022-09-12
1,2022-09-12 05:43:02+00:00,19631,Twitter for iPhone,@cb_doge @Tesla @mayemusk Still doing same thi...,2022-09-12
2,2022-09-12 04:19:57+00:00,9221,Twitter for iPhone,Looks good to roll out to all Tesla owners wit...,2022-09-12
3,2022-09-12 03:25:03+00:00,944,Twitter for iPhone,@Tesla__Mania @WholeMarsBlog That is probably ...,2022-09-12
4,2022-09-12 01:48:49+00:00,3710,Twitter for iPhone,@WholeMarsBlog Real-world validation &amp; bil...,2022-09-12


In [ ]:
# precprcess tweet
def preprocess_tweets(raw_tweet):
    
    tweet_words = []

    for word in raw_tweet.split(' '):
      if word.startswith('@') and len(word) > 1:
          word = '@user'
    
      elif word.startswith('http'):
          word = "http"
      tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)
    return tweet_proc

tweet['clean_tweet'] = tweet['Tweets'].apply(preprocess_tweets)
tweet.head(n=100)


,Date Created,Number of Likes,Source of Tweet,Tweets,date,clean_tweet
0,2022-09-12 05:44:11+00:00,1524,Twitter for iPhone,@teslaownersSV @cb_doge @Tesla @mayemusk I gue...,2022-09-12,@user @user @user @user I guess we could make ...
1,2022-09-12 05:43:02+00:00,19631,Twitter for iPhone,@cb_doge @Tesla @mayemusk Still doing same thi...,2022-09-12,"@user @user @user Still doing same thing, but ..."
2,2022-09-12 04:19:57+00:00,9221,Twitter for iPhone,Looks good to roll out to all Tesla owners wit...,2022-09-12,Looks good to roll out to all Tesla owners wit...
3,2022-09-12 03:25:03+00:00,944,Twitter for iPhone,@Tesla__Mania @WholeMarsBlog That is probably ...,2022-09-12,@user @user That is probably the right order o...
4,2022-09-12 01:48:49+00:00,3710,Twitter for iPhone,@WholeMarsBlog Real-world validation &amp; bil...,2022-09-12,@user Real-world validation &amp; billions of ...
...,...,...,...,...,...,...
95,2022-08-28 23:51:49+00:00,55584,Twitter for iPhone,This will happen again – just a matter of time...,2022-08-28,This will happen again – just a matter of time...
96,2022-08-28 23:35:36+00:00,51423,Twitter for iPhone,Godspeed Artemis!,2022-08-28,Godspeed Artemis!
97,2022-08-28 22:56:35+00:00,2832,Twitter for iPhone,@RenataKonkoly @Kristennetten @culturaltutor h...,2022-08-28,@user @user @user http
98,2022-08-28 22:54:36+00:00,3259,Twitter for iPhone,@RenataKonkoly @Kristennetten @culturaltutor T...,2022-08-28,@user @user @user The swoop of the X is meant ...


In [ ]:
#applying roberta model
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

score_list =[]
labels = ['Negative', 'Neutral', 'Positive']

for n, t in enumerate(tweet['clean_tweet']):
        encoded_tweet = tokenizer(t, return_tensors='pt')

        # sentiment analysis
        output = model(**encoded_tweet)
        #obtain probability score of sentiment using soft max
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)

        for i in range(len(scores)):
    
          l = labels[i]
          s = scores[i]

        score_list.append(scores)

        # for label, score in zip(labels, scores):
        #     tweet[label][i] = score

score_df = pd.DataFrame(score_list, columns = ['Negative', 'Neutral', 'Positive'])
tweet = pd.concat([tweet, score_df], axis=1)
tweet.head()

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

,Date Created,Number of Likes,Source of Tweet,Tweets,date,clean_tweet,Negative,Neutral,Positive
0,2022-09-12 05:44:11+00:00,1524,Twitter for iPhone,@teslaownersSV @cb_doge @Tesla @mayemusk I gue...,2022-09-12,@user @user @user @user I guess we could make ...,0.121137,0.804649,0.074213
1,2022-09-12 05:43:02+00:00,19631,Twitter for iPhone,@cb_doge @Tesla @mayemusk Still doing same thi...,2022-09-12,"@user @user @user Still doing same thing, but ...",0.057229,0.785040,0.157731
2,2022-09-12 04:19:57+00:00,9221,Twitter for iPhone,Looks good to roll out to all Tesla owners wit...,2022-09-12,Looks good to roll out to all Tesla owners wit...,0.001163,0.017109,0.981728
3,2022-09-12 03:25:03+00:00,944,Twitter for iPhone,@Tesla__Mania @WholeMarsBlog That is probably ...,2022-09-12,@user @user That is probably the right order o...,0.011641,0.439234,0.549126
4,2022-09-12 01:48:49+00:00,3710,Twitter for iPhone,@WholeMarsBlog Real-world validation &amp; bil...,2022-09-12,@user Real-world validation &amp; billions of ...,0.009935,0.352148,0.637917


In [ ]:
#tweet.to_csv('tweet_with_sent.csv')
df = pd.read_csv('tweet_with_sent.csv')
df.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweets,date,clean_tweet,Negative,Neutral,Positive
0,0,2022-09-12 05:44:11+00:00,1524,Twitter for iPhone,@teslaownersSV @cb_doge @Tesla @mayemusk I gue...,2022-09-12,@user @user @user @user I guess we could make ...,0.121137,0.804649,0.074213
1,1,2022-09-12 05:43:02+00:00,19631,Twitter for iPhone,@cb_doge @Tesla @mayemusk Still doing same thi...,2022-09-12,"@user @user @user Still doing same thing, but ...",0.057229,0.785040,0.157731
2,2,2022-09-12 04:19:57+00:00,9221,Twitter for iPhone,Looks good to roll out to all Tesla owners wit...,2022-09-12,Looks good to roll out to all Tesla owners wit...,0.001163,0.017109,0.981728
3,3,2022-09-12 03:25:03+00:00,944,Twitter for iPhone,@Tesla__Mania @WholeMarsBlog That is probably ...,2022-09-12,@user @user That is probably the right order o...,0.011641,0.439234,0.549126
4,4,2022-09-12 01:48:49+00:00,3710,Twitter for iPhone,@WholeMarsBlog Real-world validation &amp; bil...,2022-09-12,@user Real-world validation &amp; billions of ...,0.009935,0.352148,0.637917


In [ ]:
#creating an aggregated dataframe
tweets_per_day = tweet.groupby(['date']
).agg( total_likes=pd.NamedAgg(column='Number of Likes',aggfunc=sum),
         total_tweets=pd.NamedAgg(column='Tweets',aggfunc=pd.Series.count),
         total_positive=pd.NamedAgg(column='Positive',aggfunc=sum),
          total_neutral = pd.NamedAgg(column='Neutral',aggfunc=sum),
      total_negative=pd.NamedAgg(column='Negative',aggfunc=sum),
    )
tweets_per_day.head()

,total_likes,total_tweets,total_positive,total_neutral,total_negative
date,,,,,
2010-06-04,6042,1,0.354882,0.547217,0.097901
2011-12-01,264,2,1.233656,0.661762,0.104582
2011-12-03,104,2,0.605527,0.842972,0.551501
2011-12-04,163,1,0.975381,0.023312,0.001306
2011-12-21,2481,7,3.181317,2.200144,1.618539


In [ ]:
#gathering Tesla stock price data
!pip install --upgrade pandas 
!pip install --upgrade pandas-datareader

import pandas_datareader as web 
tesla_stock_price = web.DataReader('TSLA',data_source='yahoo',start = '2012-01-01',end = '2022-09-12' ) 
tesla_stock_price.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 7.6 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 9.3 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-01-03,1.966667,1.843333,1.929333,1.872000,13921500.0,1.872000
2012-01-04,1.911333,1.833333,1.880667,1.847333,9451500.0,1.847333
2012-01-05,1.862000,1.790000,1.850667,1.808000,15082500.0,1.808000
2012-01-06,1.852667,1.760667,1.813333,1.794000,14794500.0,1.794000
2012-01-09,1.832667,1.741333,1.800000,1.816667,13455000.0,1.816667


In [ ]:
tweet_and_stockprice_per_day = pd.merge(tweets_per_day, tesla_stock_price, left_index=True, right_index=True, how='right')
tweet_and_stockprice_per_day.head()

,total_likes,total_tweets,total_positive,total_neutral,total_negative,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,,,,,
2012-01-03,80.0,3.0,2.040816,0.736954,0.222230,1.966667,1.843333,1.929333,1.872000,13921500.0,1.872000
2012-01-04,NaN,NaN,NaN,NaN,NaN,1.911333,1.833333,1.880667,1.847333,9451500.0,1.847333
2012-01-05,NaN,NaN,NaN,NaN,NaN,1.862000,1.790000,1.850667,1.808000,15082500.0,1.808000
2012-01-06,74.0,3.0,1.082935,1.608398,0.308666,1.852667,1.760667,1.813333,1.794000,14794500.0,1.794000
2012-01-09,NaN,NaN,NaN,NaN,NaN,1.832667,1.741333,1.800000,1.816667,13455000.0,1.816667


In [ ]:
tweet_and_stockprice_per_day.to_csv('tweet_and_stockprice_per_day.csv')

In [ ]:
tweet_and_stockprice_per_day.fillna(0, inplace=True) # filling na values on days when there was no tweet
tweet_and_stockprice_per_day['prev_close_price'] = tweet_and_stockprice_per_day['Close'].shift(1)
tweet_and_stockprice_per_day.head(20)

,total_likes,total_tweets,total_positive,total_neutral,total_negative,High,Low,Open,Close,Volume,Adj Close,prev_close_price
Date,,,,,,,,,,,,
2012-01-03,80.0,3.0,2.040816,0.736954,0.222230,1.966667,1.843333,1.929333,1.872000,13921500.0,1.872000,NaN
2012-01-04,0.0,0.0,0.000000,0.000000,0.000000,1.911333,1.833333,1.880667,1.847333,9451500.0,1.847333,1.872000
2012-01-05,0.0,0.0,0.000000,0.000000,0.000000,1.862000,1.790000,1.850667,1.808000,15082500.0,1.808000,1.847333
2012-01-06,74.0,3.0,1.082935,1.608398,0.308666,1.852667,1.760667,1.813333,1.794000,14794500.0,1.794000,1.808000
2012-01-09,0.0,0.0,0.000000,0.000000,0.000000,1.832667,1.741333,1.800000,1.816667,13455000.0,1.816667,1.794000
2012-01-10,0.0,0.0,0.000000,0.000000,0.000000,1.850667,1.816667,1.829333,1.841333,10077000.0,1.841333,1.816667
2012-01-11,305.0,1.0,0.870379,0.126651,0.002970,1.892000,1.820000,1.841333,1.882000,10084500.0,1.882000,1.841333
2012-01-12,171.0,3.0,1.823798,1.009732,0.166470,1.908000,1.854000,1.898667,1.883333,10939500.0,1.883333,1.882000
2012-01-13,15.0,1.0,0.915007,0.083686,0.001306,1.900000,1.509333,1.893333,1.519333,82506000.0,1.519333,1.883333


In [ ]:
#removing Nan values 
tweet_and_stockprice_per_day.fillna(0, inplace=True)

In [ ]:
#checking Nan values
tweet_and_stockprice_per_day.isna().sum()

total_likes         0
total_tweets        0
total_positive      0
total_neutral       0
total_negative      0
High                0
Low                 0
Open                0
Close               0
Volume              0
Adj Close           0
prev_close_price    0
dtype: int64

In [8]:
from sklearn.linear_model import LinearRegression

In [ ]:
#base case with using only previous close price
X = tweet_and_stockprice_per_day[['prev_close_price']]
y = tweet_and_stockprice_per_day['Close']
regr.fit(X,y)
regr.score(X,y)

0.997819178202985

In [ ]:
#test case with sentiment included
X = tweet_and_stockprice_per_day[['total_likes', 'total_positive', 'total_negative','total_neutral', 'prev_close_price']]
y = tweet_and_stockprice_per_day['Close']

In [ ]:
regr = LinearRegression()
regr.fit(X, y)

LinearRegression()

In [ ]:
regr.score(X,y)

0.9978378795371413